# Managing Data from Relational Databases

In [1]:
pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sqlalchemy

In [3]:
sqlalchemy.__version__ 

'1.3.20'

In [6]:
from sqlalchemy import *

The create_engine() function takes the database as one argument. Using the code given below, we can create a database.

In [11]:
#from sqlalchemy import create_engine
engine = create_engine('sqlite:///college.db', echo = True)

An object of MetaData class from SQLAlchemy Metadata is a collection of Table objects and their associated schema constructs. It holds a collection of Table objects as well as an optional binding to an Engine or Connection.

In [12]:
#from sqlalchemy import MetaData
meta = MetaData()

## Creating a Table

In [16]:
#from sqlalchemy import Table, Column, Integer, String, MetaData
students = Table(
   'student', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String), 
   Column('contact',Integer)
)

In [17]:
meta.create_all(engine)

2021-08-19 10:20:59,954 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("students")
2021-08-19 10:20:59,955 INFO sqlalchemy.engine.base.Engine ()
2021-08-19 10:20:59,958 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("student")
2021-08-19 10:20:59,962 INFO sqlalchemy.engine.base.Engine ()
2021-08-19 10:20:59,964 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("student")
2021-08-19 10:20:59,964 INFO sqlalchemy.engine.base.Engine ()
2021-08-19 10:20:59,964 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE student (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	lastname VARCHAR, 
	contact INTEGER, 
	PRIMARY KEY (id)
)


2021-08-19 10:20:59,972 INFO sqlalchemy.engine.base.Engine ()
2021-08-19 10:21:00,382 INFO sqlalchemy.engine.base.Engine COMMIT


## Inserting a row in Database Table

In [19]:
ins = students.insert()

In [20]:
str(ins)

'INSERT INTO student (id, name, lastname, contact) VALUES (:id, :name, :lastname, :contact)'

In [21]:
ins = students.insert().values(name = 'Zarana',lastname='Gajjar',contact=1234567893)

In [23]:
conn = engine.connect()
result = conn.execute(ins)

2021-08-19 10:28:06,241 INFO sqlalchemy.engine.base.Engine INSERT INTO student (name, lastname, contact) VALUES (?, ?, ?)
2021-08-19 10:28:06,241 INFO sqlalchemy.engine.base.Engine ('Zarana', 'Gajjar', 1234567893)
2021-08-19 10:28:06,292 INFO sqlalchemy.engine.base.Engine COMMIT


In [24]:
ins = students.insert().values(name = 'Ankita', lastname = 'Tiwari')
result = conn.execute(ins)

2021-08-19 10:28:49,138 INFO sqlalchemy.engine.base.Engine INSERT INTO student (name, lastname) VALUES (?, ?)
2021-08-19 10:28:49,147 INFO sqlalchemy.engine.base.Engine ('Ankita', 'Tiwari')
2021-08-19 10:28:49,237 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
conn.execute(students.insert(), [
   {'name':'Pooja', 'lastname' : 'Joshi'},
   {'name':'Rupal','lastname' : 'Kapoor'},
   {'name':'Nilam','lastname' : 'Shah'},
   {'name':'Anjana','lastname' : 'Mehta'},
])

2021-08-19 10:29:59,975 INFO sqlalchemy.engine.base.Engine INSERT INTO student (name, lastname) VALUES (?, ?)
2021-08-19 10:29:59,982 INFO sqlalchemy.engine.base.Engine (('Pooja', 'Joshi'), ('Rupal', 'Kapoor'), ('Nilam', 'Shah'), ('Anjana', 'Mehta'))
2021-08-19 10:29:59,984 INFO sqlalchemy.engine.base.Engine COMMIT


## Select Query

In [26]:
s = students.select()
conn = engine.connect()
result = conn.execute(s)

for row in result:
    print(row)

2021-08-19 10:31:17,624 INFO sqlalchemy.engine.base.Engine SELECT student.id, student.name, student.lastname, student.contact 
FROM student
2021-08-19 10:31:17,624 INFO sqlalchemy.engine.base.Engine ()
(1, 'Zarana', 'Gajjar', 1234567893)
(2, 'Ankita', 'Tiwari', None)
(3, 'Pooja', 'Joshi', None)
(4, 'Rupal', 'Kapoor', None)
(5, 'Nilam', 'Shah', None)
(6, 'Anjana', 'Mehta', None)


In [30]:
# Where clause
s = students.select().where(students.c.id>3)
result = conn.execute(s)

for row in result:
    print (row)

2021-08-19 10:33:39,713 INFO sqlalchemy.engine.base.Engine SELECT student.id, student.name, student.lastname, student.contact 
FROM student 
WHERE student.name = ?
2021-08-19 10:33:39,717 INFO sqlalchemy.engine.base.Engine ('zarana',)


## Update Expression

stmt=students.update().where(students.c.lastname=='Kapoor').values(lastname='Kapur')

In [31]:
conn = engine.connect()
stmt=students.update().where(students.c.lastname=='Kapoor').values(lastname='Patel')
conn.execute(stmt)

s = students.select()
conn.execute(s).fetchall()

2021-08-19 10:35:01,201 INFO sqlalchemy.engine.base.Engine UPDATE student SET lastname=? WHERE student.lastname = ?
2021-08-19 10:35:01,202 INFO sqlalchemy.engine.base.Engine ('Patel', 'Kapoor')
2021-08-19 10:35:01,246 INFO sqlalchemy.engine.base.Engine COMMIT
2021-08-19 10:35:01,320 INFO sqlalchemy.engine.base.Engine SELECT student.id, student.name, student.lastname, student.contact 
FROM student
2021-08-19 10:35:01,320 INFO sqlalchemy.engine.base.Engine ()


[(1, 'Zarana', 'Gajjar', 1234567893),
 (2, 'Ankita', 'Tiwari', None),
 (3, 'Pooja', 'Joshi', None),
 (4, 'Rupal', 'Patel', None),
 (5, 'Nilam', 'Shah', None),
 (6, 'Anjana', 'Mehta', None)]

## Delete Expression

stmt = students.delete().where(students.c.id > 2)

In [34]:
conn = engine.connect()
stmt = students.delete().where(students.c.name == 'Nilam')
conn.execute(stmt)
#s = students.select()
#conn.execute(s).fetchall()

2021-08-19 11:15:17,573 INFO sqlalchemy.engine.base.Engine DELETE FROM student WHERE student.name = ?
2021-08-19 11:15:17,574 INFO sqlalchemy.engine.base.Engine ('Nilam',)
2021-08-19 11:15:17,577 INFO sqlalchemy.engine.base.Engine COMMIT
